In [ ]:
import numpy as np
np.random.seed(0)
from functions import *
#from functions import create_blank_text_file
import time
#below is saved for my own reminder of what each setting does

"""init_nuc_num = 10000 # Initial number of each nucleobase
cleav_prop = 0.4 # Probability of breaking bonds in unstructured regions
cleav_prop_struct = 0.02 # Probability of breaking bonds in structured regions
length_threshold = 10 # Strands longer than this we will checked for structure
n_iterations = 100 # Number of iterations
progress_report_freq  = 10 # Number of iterations"""

#below are the variables we manipulate
init_nuc_num = 10000 
cleav_prop = 0.5
cleav_prop_struct = 0.2
length_threshold = 15 
n_iterations = 100
progress_report_freq  = 250

#below are string versions of the variables used - this is written to our output file to help us track what settings produced the sequences
init_nuc_num_str = "*init_nuc_num = " + str(init_nuc_num) + "\n"
cleav_prop_str = "*cleav_prop = " + str(cleav_prop) + "\n"
cleav_prop_struct_str = "*cleav_prop_struct = " + str(cleav_prop_struct) + "\n"
length_threshold_str = "*length_threshold = " + str(length_threshold) + "\n"
n_iterations_str = "*n_iterations = " + str(n_iterations) + "\n"
progress_report_freq_str = "*progress_report_freq = " + str(progress_report_freq) + "\n"
settings_used = init_nuc_num_str + cleav_prop_str + cleav_prop_struct_str +  \
    length_threshold_str +  n_iterations_str + progress_report_freq_str + "\n"


#the below are used to create the file names
current_time_struct = time.localtime()
formatted_time = str(time.strftime("%Y-%m-%d %H:%M", current_time_struct))
base_directory = "./output/"
folder_name = formatted_time



#!!!!!!! THE BELOW FUNCTION IS DEFINED WITHIN FUNCTIONS.PY, BUT FOR SOME REASON I CAN NOT 
#IMPORT IT. IT SAYS THE IMPORT CAN NOT BE RESOLVED? EVERY OTHER FUNCTION FROM THAT FILE WORKS...
def create_blank_text_file(base_directory, folder_name):
    """
    Creates a new text file with a unique name based on time function was run
    Saves it to the location on the harddrive specified by base_directory
    """

    try:
        if not os.path.exists("./output/"):
            os.mkdir("./output/")
        folder_name = f"{folder_name}"
        output_folder = os.path.join(base_directory, folder_name)
        os.mkdir(output_folder)
        file_name = f"run_{formatted_time}.txt"
        file_path = os.path.join(output_folder, file_name)
        with open(file_path, "w"):
            pass

        return file_path
  
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return False

# A general note: Here we use numpy array of dtype=object because Python
# can support arbitrarily large integers. In contrast, NumPy int64 type
# will overflow for long sequences
nucleotide_list = np.array([1, 2, 3, 4] * init_nuc_num, dtype=object)
# Mapping from integer numbers to nucleobase names
mapping = {"1": "A", "2": "G", "3": "C", "4": "U"}

# Initial Phase 1
# Shuffle the nucleobases, then pair them
# We represent a nucleic acid strand by a sequence of integers (e.g. 11 is AA)
# So multiply a number by 10 and to another number to create a dimer
np.random.shuffle(nucleotide_list)
size = nucleotide_list.size
nucleotide_list = nucleotide_list[:size//2] + 10 * nucleotide_list[size//2:]

# Initial Phase 3
# Randomly break dimer bonds to create a set of monomers and dimers
cleave = np.random.random(nucleotide_list.size) > cleav_prop
cleaved = nucleotide_list[cleave]
uncleaved = nucleotide_list[~cleave]

cleaved_1 = cleaved//10
cleaved_2 = cleaved%10

nucleotide_list = np.concatenate((uncleaved, cleaved_1, cleaved_2))


#2 lines below blurred for now
 #######
#file_to_write_to = "" #MODIFIED RN
#file_to_write_to.write("*SETTINGS USED IN THE BELOW ITERATION: " + "\n" + settings_used + "\n")


for it in range(1, n_iterations + 1):
    print(it)
    

    # Phase 1: Pair nucleotide strands
    # Shuffle list of nucleotide strands
    np.random.shuffle(nucleotide_list)

    # Take the first half of the strands and calculate their lengths
    size = nucleotide_list.size
    # Use dtype=object to keep arbitrary integer prevision
    order = order_of_mag(nucleotide_list[:size//2]).astype(object)

    # If we have an even number, pair all of them
    if size%2 == 0:
        nucleotide_list = nucleotide_list[:size//2] + 10**order * nucleotide_list[size//2:]
    # If we have an odd number, do not pair last strand
    else:
        nucleotide_list_temp = nucleotide_list[:size//2] + 10**order * nucleotide_list[size//2:-1]
        nucleotide_list = np.hstack((nucleotide_list_temp, nucleotide_list[-1]))

    # Phase 2: Determine folded structures and randomly break bonds in long strands
    order = order_of_mag(nucleotide_list).astype(object)
    mono = nucleotide_list[order == 1]
    short = nucleotide_list[np.logical_and(order > 1, order < length_threshold)]
    long = nucleotide_list[order >= length_threshold]

    long = break_long(long, cleav_prop, cleav_prop_struct, mapping)

    # Phase 3: Randomly break bonds in short strands
    short = break_short(short, cleav_prop)

    nucleotide_list = np.concatenate((mono, short, long))



    if it%n_iterations == 0:
        file_path = create_blank_text_file(base_directory, folder_name)
        file_to_write_to = open(file_path, "a")
        file_to_write_to.write(settings_used)
        for s in summary_s:
            file_to_write_to.write(s + "\n")
        


    # Print progress report: show sequences of 10 longest strands


    elif it%progress_report_freq==0:
        print("\nRESULTS\n")



        summary = np.sort(nucleotide_list)[::-1] #to get all lengths, remove [::-1][-:10]
        print(summary)
        summary_s = convert_int_to_str_seq(summary, mapping)
        #file_to_write_to.write("\n")
        for s in summary_s:
            print(s)
            #file_to_write_to.write(s + "\n")


    #file_to_write_to.close()



#generate_sequence_length_histogram('test_file_1.txt')


            
